In [2]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

## defining parameterized operations that are repeated num_time_steps times

In [3]:
N_cav = 10

In [4]:
def qubit_unitary(alpha):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    return expm(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conjugate() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [5]:
def qubit_cavity_unitary(beta):
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
        )
        / 2
    )

In [6]:
alpha = 0.1 + 0.1j
beta = 0.1 + 0.1j
Uq = qubit_unitary(alpha)
Uqc = qubit_cavity_unitary(beta)
print(
    "Uq unitary check:",
    jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
)
print(
    "Uqc unitary check:",
    jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
)

Uq unitary check: True
Uqc unitary check: True


In [7]:
qubit_unitary(0.1)

Array([[0.99875026+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        -0.04997917j, 0.99875026+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j 

In [8]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, gamma, delta):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

### defining initial (thermal) state

In [9]:
# initial state is a thermal state coupled to a qubit in the ground state?
n_average = 1
# natural logarithm
beta = jnp.log((1 / n_average) + 1)
diags = jnp.exp(-beta * jnp.arange(N_cav))
normalized_diags = diags / jnp.sum(diags, axis=0)
rho_cav = jnp.diag(normalized_diags)

In [10]:
rho_cav.shape

(10, 10)

In [11]:
rho0 = tensor(rho_cav, basis(2, 0) @ basis(2, 0).conj().T)

In [12]:
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho0, 1, povm_measure_operator, [0.1, -3 * jnp.pi / 2]
)

Array(0.95971493, dtype=float64)

### defining target state

In [13]:
psi_target = tensor(
    (fock(N_cav, 1) + fock(N_cav, 2) + fock(N_cav, 3)) / jnp.sqrt(3), basis(2)
)

rho_target = psi_target @ psi_target.conj().T
rho_target.shape

(20, 20)

In [14]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho0, C_target=rho_target, type="density"))

0.3820679100780015


### initialize random params

In [ ]:
num_time_steps = 5
num_of_iterations = 1000
learning_rate = 0.001
# avg_photon_numer = 2 When testing kitten state


initial_params = {
    "POVM": [jnp.pi / 3, jnp.pi / 3],
    "U_q": [jnp.pi / 3],
    "U_qc": [jnp.pi / 3],
}


result = optimize_pulse_with_feedback(
    U_0=rho0,
    C_target=rho_target,
    parameterized_gates=[
        povm_measure_operator,
        qubit_unitary,
        qubit_cavity_unitary,
    ],
    measurement_indices=[0],
    initial_params=initial_params,
    num_time_steps=num_time_steps,
    mode="lookup",
    goal="fidelity",
    optimizer="adam",
    max_iter=num_of_iterations,
    convergence_threshold=1e-20,
    learning_rate=learning_rate,
    type="density",
    batch_size=1,
)

Iteration 0, Loss: 0.126444
Iteration 10, Loss: 0.591464
Iteration 20, Loss: 0.210558
Iteration 30, Loss: 0.216585
Iteration 40, Loss: 0.151532
Iteration 50, Loss: 0.477789
Iteration 60, Loss: 0.836576
Iteration 70, Loss: 0.052821
Iteration 80, Loss: 0.876998
Iteration 90, Loss: 0.746338
Iteration 100, Loss: 0.061098
Iteration 110, Loss: 0.531677
Iteration 120, Loss: 0.604752
Iteration 130, Loss: 0.156887
Iteration 140, Loss: 0.574097
Iteration 150, Loss: 0.275747
Iteration 160, Loss: 0.181457
Iteration 170, Loss: 0.167959
Iteration 180, Loss: 0.171134
Iteration 190, Loss: 0.149490
Iteration 200, Loss: 0.168637
Iteration 210, Loss: 0.332203
Iteration 220, Loss: 1.177094
Iteration 230, Loss: 0.086570
Iteration 240, Loss: 0.932464
Iteration 250, Loss: 0.144428
Iteration 260, Loss: 0.885863
Iteration 270, Loss: 0.334442
Iteration 280, Loss: 0.165531
Iteration 290, Loss: 0.689979
Iteration 300, Loss: 0.183802
Iteration 310, Loss: 0.259053
Iteration 320, Loss: 0.452697
Iteration 330, Loss: 

In [16]:
result

FgResult(optimized_trainable_parameters=[[Array([1.40694074, 0.6653006 , 3.57390182, 0.29480887], dtype=float64), Array([ 1.00949408e-03,  9.42752810e-01, -1.44330754e-01,  2.21584228e+00],      dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64), Array([0., 0., 0., 0.], dtype=float64)], [Array([1.5286672 , 0.07872084, 2.84378064, 3.14850513], dtype=float64), Array([-1.45899795,  1.76742495, -1.04845808,  0.73274539], dtype=float64), Array([ 2.22584508, -0.80841946,  3.21674693,  2.35703114], dtype=float64), Arr

In [17]:
print(result.final_purity)

None


In [18]:
print(result.final_fidelity)

0.23745280227685733


In [19]:
from feedback_grape.utils.fidelity import fidelity

print("initial fidelity:", fidelity(C_target=rho_target, U_final=rho0, type="density"))
for i, state in enumerate(result.final_state):
    print(f"fidelity of state {i}:", fidelity(C_target=rho_target, U_final=state, type="density"))

initial fidelity: 0.3820679100780015
fidelity of state 0: 0.23745280227685733


In [20]:
result.returned_params

[[Array([[1.04719755, 1.04719755]], dtype=float64),
  Array([[-0.14433075]], dtype=float64),
  Array([[2.21584228]], dtype=float64)],
 [Array([[0.00100949, 0.94275281]], dtype=float64),
  Array([[3.21674693]], dtype=float64),
  Array([[2.35703114]], dtype=float64)],
 [Array([[ 2.22584508, -0.80841946]], dtype=float64),
  Array([[2.81592071]], dtype=float64),
  Array([[1.92996728]], dtype=float64)],
 [Array([[ 3.35110143, -2.22954514]], dtype=float64),
  Array([[0.11174841]], dtype=float64),
  Array([[3.00641162]], dtype=float64)],
 [Array([[-0.86931601,  1.30901033]], dtype=float64),
  Array([[2.16786601]], dtype=float64),
  Array([[2.39788663]], dtype=float64)]]

In [21]:
from feedback_grape.utils.povm import povm
from feedback_grape.fgrape import apply_gate
from feedback_grape.utils.purity import purity
import jax

time_steps = 5

rho = rho0
print(
    "initial purity:",
    fidelity(U_final=rho, C_target=rho_target, type="density"),
)
time_step_keys = jax.random.split(jax.random.PRNGKey(2), time_steps)
print("time step keys:", time_step_keys.shape)
for i in range(time_steps):
    params = result.returned_params[i][0]
    rho, _, _ = povm(rho, povm_measure_operator, params[0], time_step_keys[i])
    rho = apply_gate(
        rho,
        qubit_unitary,
        params[1],
        type="density",
    )
    rho = apply_gate(
        rho,
        qubit_cavity_unitary,
        params[2],
        type="density",
    )
    print(
        f"fid of rho after time step {i}",
        fidelity(U_final=rho, C_target=rho_target, type="density"),
    )
final_rho_cav = rho

initial purity: 0.3820679100780015
time step keys: (5, 2)


TypeError: qubit_unitary() takes 1 positional argument but 2 were given

In [ ]:
print(result.iterations)

In [ ]:
print(result.returned_params[1])

In [ ]:
import jax

F = []
F_1_0 = jnp.array([jnp.pi / 2, 0])
key = jax.random.PRNGKey(0)
appo = jax.random.uniform(key, shape=(2**1, 4)) * jnp.pi
F.append(appo)
F

In [ ]:
variables = [F] + [F_1_0]

In [ ]:
variables

In [ ]:
def prepare_parameters_from_dict(params_dict):
    """
    Convert a nested dictionary of parameters to a flat list and record shapes.

    Args:
        params_dict: Nested dictionary of parameters.

    Returns:
        tuple: Flattened parameters list and list of shapes.
    """
    flat_params = []
    param_shapes = []

    # returns a flat list of the leaves
    def flatten_dict(d):
        result = []
        for key, value in d.items():
            if isinstance(value, dict):
                result.extend(flatten_dict(value))
            else:
                result.append(value)
        return result

    # flatten each top-level gate
    for gate_name, gate_params in params_dict.items():
        if isinstance(gate_params, dict):
            # Extract parameters for this gate
            gate_flat_params = jnp.array(flatten_dict(gate_params))
        else:
            # If already a flat array
            gate_flat_params = jnp.array(gate_params)
        # this is checking if use can enter sth like {'gate1': 1} instead of {"gate1": {"param1": 1}}
        # if not (isinstance(gate_flat_params, list)):
        #     flat_params.append([gate_flat_params])
        #     param_shapes.append(1)
        # else:
        #     flat_params.append(gate_flat_params)
        #     param_shapes.append(len(gate_flat_params))
        flat_params.append(gate_flat_params)
        param_shapes.append(gate_flat_params.shape)
    return flat_params, param_shapes

In [ ]:
import numpy as np


def reshape_params(param_shapes, rnn_flattened_params):
    """
    Reshape the parameters for the gates.
    """
    # Reshape the flattened parameters from RNN output according
    # to each gate corressponding params
    reshaped_params = []
    param_idx = 0
    for shape in param_shapes:
        num_params = int(np.prod(shape))
        # rnn outputs a flat list, this takes each and assigns according to the shape
        gate_params = rnn_flattened_params[
            param_idx : param_idx + num_params
        ].reshape(shape)
        reshaped_params.append(gate_params)
        param_idx += num_params

    new_params = reshaped_params
    return new_params

In [ ]:
import os

os.sys.path.append("..")
import jax
import jax.numpy as jnp
from feedback_grape.fgrape import prepare_parameters_from_dict, reshape_params

initial_params = {
    "POVM": [jnp.pi / 3, jnp.pi / 3],
    "U_q": [jnp.pi / 3],
    "U_qc": [jnp.pi / 3],
}

flat_params, param_shapes = prepare_parameters_from_dict(initial_params)
num_of_columns = 4
num_of_sub_lists = 3
F = []


def construct_ragged_row(num_of_rows):
    res = []
    for i in range(num_of_rows):
        flattened = jax.random.uniform(
            jax.random.PRNGKey(0 + i),
            shape=(num_of_columns,),
            minval=-jnp.pi,
            maxval=jnp.pi,
        )
        res.append(flattened)
    return res


for i in range(1, num_of_sub_lists + 1):
    F.append(construct_ragged_row(num_of_rows=2**i))

for i in range(len(F)):
    print("length of F[{}]: {}".format(i, len(F[i])))

print("############")
min_num_of_rows = 2 ** len(F)
for i in range(len(F)):
    if len(F[i]) < min_num_of_rows:
        zeros_arrays = [
            jnp.zeros((num_of_columns,), dtype=jnp.float32)
            for _ in range(min_num_of_rows - len(F[i]))
        ]
        F[i] = F[i] + zeros_arrays

print("############")
from pprint import pprint

for i in range(len(F[0])):
    print("length of F[{}]: {}".format(i, len(F[0][i])))
    pprint(F[0][i])

In [ ]:
print(F)

In [ ]:
def convert_to_index(measurement_history):
    # Convert measurement history from [1, -1, ...] to [0, 1, ...] and then to an integer index
    binary_history = jnp.where(jnp.array(measurement_history) == 1, 0, 1)
    print(f"binary_history: {binary_history}")
    # Convert binary list to integer index (e.g., [0,1] -> 1)
    # Reverse the binary_history to operate from last element backwards
    reversed_binary = binary_history[::-1]
    int_index = sum(
        (2**i) * reversed_binary[i] for i in range(len(reversed_binary))
    )
    return int_index


def extract_from_lut(lut, measurement_history):
    """
    Extract parameters from the lookup table based on the measurement history.

    Args:
        lut: Lookup table for parameters.
        measurement_history: History of measurements.
        time_step: Current time step.

    Returns:
        Extracted parameters.
    """
    sub_array_idx = len(measurement_history) - 1
    print(f"sub_array_idx: {sub_array_idx}")
    sub_array_param_idx = convert_to_index(measurement_history)
    print(f"sub_array_param_idx: {sub_array_param_idx}")
    return lut[sub_array_idx][sub_array_param_idx]


extracted_lut_params = extract_from_lut(F, [1, -1])

In [ ]:
print(extracted_lut_params)

In [ ]:
import tensorflow as tf

print(tf.random.uniform(shape=[1]))

In [ ]:
import jax

jax.random.uniform(
    jax.random.PRNGKey(0),
    shape=(1,),
)